1 Для нашего пайплайна (Case1) поэкспериментировать с разными моделями:

 -бустинг
 
 -логистическая регрессия
 
2 Отобрать лучшую модель по метрикам (какая по вашему мнению здесь наиболее подходящая ML-метрика)

3 Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как   в вопросе 2:

  -доллар на удержание
  
  -доллара - с каждого правильно классифицированного (True Positive)
  
*Провести подбор гиперпараметров лучшей модели по итогам 2-3

*Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91' -O churn_data.csv

--2022-03-17 19:21:16--  https://drive.google.com/uc?export=download&id=1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91
Распознаётся drive.google.com (drive.google.com)... 64.233.162.194, 2a00:1450:4010:c05::c2
Подключение к drive.google.com (drive.google.com)|64.233.162.194|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа... 303 See Other
Адрес: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9e6kjtnmsenq72hln2vnn0ofvcfbqere/1647534075000/14904333240138417226/*/1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91?e=download [переход]
Предупреждение: в HTTP шаблоны не поддерживаются.
--2022-03-17 19:21:17--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9e6kjtnmsenq72hln2vnn0ofvcfbqere/1647534075000/14904333240138417226/*/1yIIxDfW7Wfq-wPlbsa0dFrSlD3r-Ai91?e=download
Распознаётся doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 142.251.1.132, 2a00:1450:4010:c1e::84
Подключение к 

In [2]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [4]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        print(test_columns)
        for col_ in self.columns:
            print(col_)
            if col_ not in test_columns:
                print('ll')
                X[col_] = 0
        return X[self.columns]

In [5]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    
    final_transformers.append((cont_col, cont_transformer))
    
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier

pipeline_lin = Pipeline([
    ('features', feats),
    ('classifier', LinearRegression()),
])

pipeline_boost = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier()),
])

# построим модели

In [8]:
pipeline_lin.fit(X_train, y_train)
pipeline_boost.fit(X_train, y_train)

['Geography_France', 'Geography_Germany', 'Geography_Spain']
Geography_France
Geography_Germany
Geography_Spain
['Gender_Female', 'Gender_Male']
Gender_Female
Gender_Male
['Tenure_0', 'Tenure_1', 'Tenure_2', 'Tenure_3', 'Tenure_4', 'Tenure_5', 'Tenure_6', 'Tenure_7', 'Tenure_8', 'Tenure_9', 'Tenure_10']
Tenure_0
Tenure_1
Tenure_2
Tenure_3
Tenure_4
Tenure_5
Tenure_6
Tenure_7
Tenure_8
Tenure_9
Tenure_10
['HasCrCard_0', 'HasCrCard_1']
HasCrCard_0
HasCrCard_1
['IsActiveMember_0', 'IsActiveMember_1']
IsActiveMember_0
IsActiveMember_1
['Geography_France', 'Geography_Germany', 'Geography_Spain']
Geography_France
Geography_Germany
Geography_Spain
['Gender_Female', 'Gender_Male']
Gender_Female
Gender_Male
['Tenure_0', 'Tenure_1', 'Tenure_2', 'Tenure_3', 'Tenure_4', 'Tenure_5', 'Tenure_6', 'Tenure_7', 'Tenure_8', 'Tenure_9', 'Tenure_10']
Tenure_0
Tenure_1
Tenure_2
Tenure_3
Tenure_4
Tenure_5
Tenure_6
Tenure_7
Tenure_8
Tenure_9
Tenure_10
['HasCrCard_0', 'HasCrCard_1']
HasCrCard_0
HasCrCard_1
['IsA

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [9]:
preds_lin = pipeline_lin.predict(X_test)
preds_boost = pipeline_boost.predict_proba(X_test)[:, 1]

['Geography_France', 'Geography_Germany', 'Geography_Spain']
Geography_France
Geography_Germany
Geography_Spain
['Gender_Female', 'Gender_Male']
Gender_Female
Gender_Male
['Tenure_0', 'Tenure_1', 'Tenure_2', 'Tenure_3', 'Tenure_4', 'Tenure_5', 'Tenure_6', 'Tenure_7', 'Tenure_8', 'Tenure_9', 'Tenure_10']
Tenure_0
Tenure_1
Tenure_2
Tenure_3
Tenure_4
Tenure_5
Tenure_6
Tenure_7
Tenure_8
Tenure_9
Tenure_10
['HasCrCard_0', 'HasCrCard_1']
HasCrCard_0
HasCrCard_1
['IsActiveMember_0', 'IsActiveMember_1']
IsActiveMember_0
IsActiveMember_1
['Geography_France', 'Geography_Germany', 'Geography_Spain']
Geography_France
Geography_Germany
Geography_Spain
['Gender_Female', 'Gender_Male']
Gender_Female
Gender_Male
['Tenure_0', 'Tenure_1', 'Tenure_2', 'Tenure_3', 'Tenure_4', 'Tenure_5', 'Tenure_6', 'Tenure_7', 'Tenure_8', 'Tenure_9', 'Tenure_10']
Tenure_0
Tenure_1
Tenure_2
Tenure_3
Tenure_4
Tenure_5
Tenure_6
Tenure_7
Tenure_8
Tenure_9
Tenure_10
['HasCrCard_0', 'HasCrCard_1']
HasCrCard_0
HasCrCard_1
['IsA

# оценка модели

In [10]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [33]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_lin)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.28797157448607524, F-Score=0.513, Precision=0.431, Recall=0.635


In [34]:
cnf_matrix_lin = confusion_matrix(y_test, preds_lin > thresholds[ix])
cnf_matrix_lin

array([[1564,  427],
       [ 187,  322]])

In [35]:
precision, recall, thresholds = precision_recall_curve(y_test, preds_boost)
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.4085078904556646, F-Score=0.646, Precision=0.704, Recall=0.597


In [36]:
cnf_matrix_boost = confusion_matrix(y_test, preds_boost > thresholds[ix])
cnf_matrix_boost

array([[1863,  128],
       [ 206,  303]])

по метрикам ml лучше справился бустинг, но нас интересуют больше recall а он у линейной модели больше. выбор модели будет зависить от стратегии компании т.е. если компания хочет как можно больше удержать клиентов не смотря на затраты то здесь подойдет линейная регрессия потому что она больше находит людей которые могут уйти, но она так же много ошибыеться и это ведет к доп расходам. боустинг ошибаеться меньше, но он и находит людей оттока тоже меньше

In [24]:
#lineal model
TN = cnf_matrix_lin[0][0]
FP = cnf_matrix_lin[0][1]
FN = cnf_matrix_lin[1][0]
TP = cnf_matrix_lin[1][1]

retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

23

In [37]:
#boost model
TN = cnf_matrix_boost[0][0]
FP = cnf_matrix_boost[0][1]
FN = cnf_matrix_boost[1][0]
TP = cnf_matrix_boost[1][1]

retain_sum = (FP + TP) * 1
income = TP * 2

income - retain_sum

175